In [ ]:
#!pip install pandas
#!pip install numpy
#pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 11.0 MB/s  0:00:00eta 0:00:01

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [20]:
df = pd.read_csv('./dataset/spotify_songs.csv')

In [21]:
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [27]:
print(len(df[df["track_id"] == "1HfMVBKM75vxSfsQ5VefZ5"]))

print(df.isnull().sum())

7
track_id                    0
track_name                  5
track_artist                5
track_popularity            0
track_album_id              0
track_album_name            5
track_album_release_date    0
playlist_name               0
playlist_id                 0
playlist_genre              0
playlist_subgenre           0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
dtype: int64


In [ ]:
df_clean = df.drop_duplicates(subset=['track_id'], keep='first')
df_clean = df_clean.dropna()

print("# rows old data: ", len(df))
print("# rows new data: ", len(df_clean))

df.to_csv('./dataset/spotify_clean_data.csv', index=False)

# rows old data:  32833
# rows new data:  28352


In [ ]:
import csv
def get_db_connection():
    conn = psycopg2.connect(
        host="localhost",          
        database="music_app_db",   
        user="myuser",             
        password="mypassword",     
        port="5432"
    )
    return conn

def import_csv_to_postgres(conn, cur, filepath): #populating db
    try:
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            next(reader) 
            for row in reader:
                cur.execute(
                """
                INSERT INTO tracks (
                    track_id, track_name, track_artist, track_popularity,
                    track_album_id, track_album_name, track_album_release_date,
                    playlist_name, playlist_id, playlist_genre, playlist_subgenre,
                    danceability, energy, key, loudness, mode, speechiness,
                    acousticness, instrumentalness, liveness, valence, tempo, duration_ms
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row[0],  # track_id
                    row[1],  # track_name
                    row[2],  # track_artist
                    row[3],  # track_popularity
                    row[4],  # track_album_id
                    row[5],  # track_album_name
                    row[6],  # track_album_release_date
                    row[7],  # playlist_name
                    row[8],  # playlist_id
                    row[9],  # playlist_genre
                    row[10], # playlist_subgenre
                    row[11], # danceability
                    row[12], # energy
                    row[13], # key
                    row[14], # loudness
                    row[15], # mode
                    row[16], # speechiness
                    row[17], # acousticness
                    row[18], # instrumentalness
                    row[19], # liveness
                    row[20], # valence
                    row[21], # tempo
                    row[22]  # duration_ms
                )
            )
        conn.commit()
        print("Data imported successfully!")

    except Exception as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            cur.close()
            conn.close()

In [6]:
print(df.columns)

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='str')


In [18]:
conn = get_db_connection()
cursor = conn.cursor()

In [19]:
sql = '''
CREATE TABLE tracks (
    track_id CHAR(22) PRIMARY KEY,
    track_name TEXT,                  
    track_artist TEXT,                
    track_popularity INT,
    track_album_id CHAR(22),
    track_album_name TEXT,            
    track_album_release_date VARCHAR(20), 
    playlist_name TEXT,               
    playlist_id VARCHAR(50),          
    playlist_genre VARCHAR(50),       
    playlist_subgenre VARCHAR(50),    
    danceability DOUBLE PRECISION,    
    energy DOUBLE PRECISION,
    key INT,
    loudness DOUBLE PRECISION,
    mode INT,
    speechiness DOUBLE PRECISION,
    acousticness DOUBLE PRECISION,    
    instrumentalness DOUBLE PRECISION,
    liveness DOUBLE PRECISION,
    valence DOUBLE PRECISION,
    tempo DOUBLE PRECISION,           
    duration_ms INT
);
'''

In [20]:
cursor.execute(sql)
conn.commit()
conn.close()

In [21]:
conn = get_db_connection()
cursor = conn.cursor()
import_csv_to_postgres(conn, cursor, "./dataset/spotify_clean_data.csv")

Data imported successfully!
